In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install transformers
!pip install stop_words
!pip install symspellpy
!pip install language_detector 
!pip install cached_property
!pip install sentencepiece
!pip install config
!pip install umap
!pip install sentence-transformers

     |████████████████████████████████| 2.5MB 15.7MB/s 
     |████████████████████████████████| 901kB 57.9MB/s 
     |████████████████████████████████| 3.3MB 45.9MB/s 
  Created wheel for stop-words: filename=stop_words-2018.7.23-cp37-none-any.whl size=32913 sha256=e493432a6fd2c8b89706f590cc39da169af1ebbba9c98d147e65a2eca29d2b51
  Stored in directory: /root/.cache/pip/wheels/75/37/6a/2b295e03bd07290f0da95c3adb9a74ba95fbc333aa8b0c7c78
Successfully built stop-words
     |████████████████████████████████| 2.6MB 15.5MB/s 
  Created wheel for language-detector: filename=language_detector-5.0.2-cp37-none-any.whl size=7054 sha256=ce7f1ef7693372c5e6c96dc697bb753538f89bdecd4bbe41040254df923bebaf
  Stored in directory: /root/.cache/pip/wheels/7d/37/fa/2098a4aa6c0d94d6ddff0d3a79669e12bc4f7baca8a760b3db
Successfully built language-detector
     |████████████████████████████████| 1.2MB 14.2MB/s 
  Created wheel for umap: filename=umap-0.1.1-cp37-none-any.whl size=3568 sha256=649accae10e222251a76418

In [4]:
root_path = "/content/drive/MyDrive/SCHOOL/PhD/Code/context-encoder-v2"
import sys, os
import config

config.root_path = os.path.abspath(root_path)
sys.path.insert(0, config.root_path)

In [5]:
import tensorflow as tf

from src.encoders.context_encoder_ldabert import ContextEncoder, ContextEncoderComplex
from src.dataset.ldabert import LDABERTDataset

from tensorflow.python import keras
import toml
import json
import pandas as pd
import numpy as np
from utils.experiments import get_experiments_json, get_experiments, save_results

## Experiment

In [6]:
dataset = LDABERTDataset(dataset_type="clinical",
                       pct_data=1,
                       max_segment_length=5,
                       augment_pct=1)

In [7]:
sentences, tokenized_sentences, labels = dataset.process(preprocess=False)

In [8]:
# vectors_path = '../data/clinical_vectors/lda_bert_{}_{}.pkl'.format(dataset_type, len(sentences))
vectors_path = '../../data/clinical_vectors/lda_bert_{}_{}_{}.pkl'.format(dataset.dataset_type, 
                                                                    dataset.pct_data, 
                                                                    dataset.augment_pct)

saved_vectors, saved_labels, saved_sentences = dataset.get_vectors(vectors_path)

if len(saved_vectors) == 0:
    saved_vectors, saved_labels, saved_sentences = dataset.create_vectors(vectors_path)

In [9]:
left_input, mid_input, right_input = dataset.format_sentences_tri_input(saved_vectors)

In [10]:
experiments_config = get_experiments_json('ldabert_complex_test')
experiments_config_df = pd.DataFrame.from_dict(experiments_config)
experiments_config_df

,bert_type,dataset_type,final_dropout,dense_neurons,lstm_size,lstm_dropout_percentage,cnn_filters,cnn_kernel_size,pool_size,pct_data,augment_pct,epochs
0,ldabert,clinical,0.2,64,256,0.2,8,3,2,1,1,1000
1,ldabert,clinical,0.2,64,256,0.5,8,3,2,1,1,1000
2,ldabert,clinical,0.2,64,256,0.8,8,3,2,1,1,1000
3,ldabert,clinical,0.2,256,256,0.2,8,3,2,1,1,1000
4,ldabert,clinical,0.2,256,256,0.5,8,3,2,1,1,1000
5,ldabert,clinical,0.2,256,256,0.8,8,3,2,1,1,1000
6,ldabert,clinical,0.5,64,256,0.2,8,3,2,1,1,1000
7,ldabert,clinical,0.5,64,256,0.5,8,3,2,1,1,1000
8,ldabert,clinical,0.5,64,256,0.8,8,3,2,1,1,1000
9,ldabert,clinical,0.5,256,256,0.2,8,3,2,1,1,1000


In [12]:
experiments_config_df.to_csv(r'../models/experiment.csv', header=None, index=None, sep=' ', mode='a')

In [15]:
def get_random_hash(k):
  import random, string
  x = ''.join(random.choices(string.ascii_letters + string.digits, k=k))
  return x

In [19]:
for experiment in experiments_config:
    bert_type = experiment['bert_type']
    dataset_type = experiment['dataset_type']
    final_dropout = experiment['final_dropout']
    dense_neurons = experiment['dense_neurons']
    lstm_size = experiment['lstm_size']
    lstm_dropout_percentage = experiment['lstm_dropout_percentage']
    cnn_filters = experiment['cnn_filters']
    cnn_kernel_size = experiment['cnn_kernel_size']
    pool_size = experiment['pool_size']
    pct_data = experiment['pct_data']
    augment_pct = experiment['augment_pct']
    epochs = experiment['epochs']
    BATCH_SIZE = 512
    random_hash = get_random_hash(5)
    print("params:", experiment)

    
    # init model
    print("initializing model...")
    model = ContextEncoder(final_dropout=final_dropout,
                            dense_neurons=dense_neurons,
                             lstm_size=lstm_size,
                             lstm_dropout_percentage=lstm_dropout_percentage,
                             cnn_filters=cnn_filters,
                             cnn_kernel_size=cnn_kernel_size,
                             pool_size=pool_size)
    
    # print("number of params: ", sum([np.prod(keras.get_value(w).shape) for w in model.trainable_weights]))
    
    # init dataset
    print("initializing dataset...")
    dataset = LDABERTDataset(dataset_type=dataset_type,
                            pct_data=pct_data,
                            max_segment_length=5,
                            augment_pct=augment_pct)
    
    # process dataset
    print("processing dataset...")
    sentences, tokenized_sentences, labels = dataset.process(preprocess=False)

    vectors_path = '../../data/{}_vectors/lda_bert_{}_{}_{}.pkl'.format(dataset.dataset_type, dataset.dataset_type, 
                                                                        dataset.pct_data, 
                                                                        dataset.augment_pct)

    saved_vectors, saved_labels, saved_sentences = dataset.get_vectors(vectors_path)

    if len(saved_vectors) == 0:
        saved_vectors, saved_labels, saved_sentences = dataset.create_vectors(vectors_path)

    left_input, mid_input, right_input = dataset.format_sentences_tri_input(saved_vectors)

    # get class weight
    neg, pos = np.bincount(labels.flatten())
    initial_bias = np.log([pos/neg])
    
    total=len(labels)
    weight_for_0 = (1 / neg)*(total)/2.0 
    weight_for_1 = (1 / pos)*(total)/2.0

    class_weight = {0: weight_for_0, 1: weight_for_1}
    print("class weight", class_weight)
    
    # create checkpoint path
    checkpoint_filepath = '{}/models/LDABERT/complex/{}/{}-{}-pct-{}-aug_{}/checkpoint.ckpt'.format(
                            dataset.dataset_type,  
                            config.root_path,                  
                            len(sentences), 
                            dataset.pct_data,
                            dataset.augment_pct,
                            random_hash)
    print(checkpoint_filepath)
    
    # get callbacks ready.
    model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
        filepath=checkpoint_filepath,
        save_weights_only=True,
        monitor='val_accuracy',
        save_best_only=False,
        mode="auto",
        save_freq="epoch")

    early_stopping = tf.keras.callbacks.EarlyStopping(
        monitor='val_accuracy', 
        verbose=1,
        patience=10,
        mode='max',
        restore_best_weights=True)

    callbacks = [
    #     early_stopping,
        model_checkpoint_callback
    ]
    
    # compiling model
    print("compiling the model...")
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
                  loss=tf.keras.losses.BinaryCrossentropy(),
                  metrics=[
                      keras.metrics.BinaryAccuracy(name='accuracy')
                  ])
    
    # try:
    #     model.load_weights(checkpoint_filepath)
    #     print("model loaded.")
    # except:
    #     print("No checkpoint available.")
    
    print("starting the training process...")
    history = model.fit([left_input, mid_input, right_input], 
                        tf.convert_to_tensor(saved_labels), 
                        epochs=epochs,
                        validation_split=0.25,
                        batch_size=BATCH_SIZE,
                        verbose=1, 
                        # class_weight=class_weight,
                        callbacks=callbacks)
    
    # assigning history to experiment object for saving.
    experiment["history"] = history.history
    experiment["hash"] = random_hash
    
    print("saving results...")
    save_results(experiment)

Streaming output truncated to the last 5000 lines.
24/24 [==============================] - 0s 5ms/step - loss: 0.0271 - accuracy: 0.9901 - val_loss: 0.5490 - val_accuracy: 0.8829
Epoch 686/1000
24/24 [==============================] - 0s 7ms/step - loss: 0.0414 - accuracy: 0.9828 - val_loss: 0.3639 - val_accuracy: 0.9013
Epoch 687/1000
24/24 [==============================] - 0s 5ms/step - loss: 0.0225 - accuracy: 0.9920 - val_loss: 0.3669 - val_accuracy: 0.9018
Epoch 688/1000
24/24 [==============================] - 0s 7ms/step - loss: 0.0262 - accuracy: 0.9905 - val_loss: 0.4042 - val_accuracy: 0.9001
Epoch 689/1000
24/24 [==============================] - 0s 7ms/step - loss: 0.0266 - accuracy: 0.9906 - val_loss: 0.3620 - val_accuracy: 0.9079
Epoch 690/1000
24/24 [==============================] - 0s 5ms/step - loss: 0.0251 - accuracy: 0.9917 - val_loss: 0.3709 - val_accuracy: 0.9051
Epoch 691/1000
24/24 [==============================] - 0s 7ms/step - loss: 0.0237 - accuracy: 0.991

KeyboardInterrupt: ignored